<a href="https://colab.research.google.com/github/theolht/NLP_Project/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import numpy as np 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop


nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [68]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [69]:
df = pd.read_csv (r'gdrive/MyDrive/NLP_Project/training.1600000.processed.noemoticon.csv', encoding='latin-1')

Just keeping the important data. The comment and the comment sentiment.

In [71]:
data = df.copy()
data.columns = ['target', 'id_tweet', 'date', 'flag', 'user', 'comment']
del data['flag']
del data['id_tweet']
del data['date']
del data['user']

data = data.sample(frac = 1)
data = data.head(100000)
data

,target,comment
969687,4,lol I think I might be spoiled. How about if...
1081789,4,finishing packing for trip to College Station ...
1043259,4,"Hey guys, I won't be using my Globe phone from..."
322271,0,Damn my computer is officially done for. how ...
1109953,4,startin my day with a run around Boone
...,...,...
1218319,4,@EASTBAYTRIBE thanks babe
1483730,4,Raffa is coming over tonight
1345628,4,@bravesgirl5 8:45am is early?.. in highschool ...
509390,0,Missin' Superman...Gotta wait another week...


Data cleaning

In [72]:
stemmer = LancasterStemmer()

for id, comment in data['comment'].items():
  comment = nltk.word_tokenize(comment)
  comment = [ ch.lower() for ch in comment if ch.isalpha() or ch == '.']
  comment = [w for w in comment if not w in list(nltk.corpus.stopwords.words('english'))]
  comment = [stemmer.stem(w) for w in comment]
  data['comment'][id] = comment

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Changing the comment sentiment from 4 to 1 for easier utilization

In [73]:
data['target'][data['target']==4]=1
data

,target,comment
969687,1,"[lol, think, might, spoil, ., giv, today, prom..."
1081789,1,"[fin, pack, trip, colleg, stat, weekend, danie..."
1043259,1,"[hey, guy, wo, us, glob, phon, today, til, sun..."
322271,0,"[damn, comput, off, don, ., im, fin, mixtap]"
1109953,1,"[startin, day, run, around, boon]"
...,...,...
1218319,1,"[eastbaytrib, thank, bab]"
1483730,1,"[raff, com, tonight]"
1345628,1,"[ear, highschool, class, start, hah]"
509390,0,"[missin, superm, got, ta, wait, anoth, week]"


RNN model

In [74]:
model = Sequential()
model.add(SimpleRNN(128, input_shape=(51,1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [75]:
X = data.comment
y = data.target

max = 500   # length max
tok = Tokenizer(num_words=2000)
tok.fit_on_texts(X)

sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max)

Xtrain, Xtest, ytrain, ytest = train_test_split(sequences_matrix, y, test_size=0.2, random_state=2)

In [76]:
Xtest = np.expand_dims(Xtest, axis=2)
loss,acc = model.evaluate(Xtest, ytest)

print("loss: %.2f" % (loss))
print("accuracy: %.2f" % (acc))

625/625 [==============================] - 19s 30ms/step - loss: 0.8070 - accuracy: 0.5028
loss: 0.81
accuracy: 0.50


In [77]:
Xtrain = np.expand_dims(Xtrain, axis=2)

In [55]:
model.fit(Xtrain,ytrain, epochs=5)

Epoch 1/5
2500/2500 [==============================] - 249s 99ms/step - loss: 0.6970 - accuracy: 0.5132
Epoch 2/5
2500/2500 [==============================] - 249s 100ms/step - loss: 0.6945 - accuracy: 0.5135
Epoch 3/5
2500/2500 [==============================] - 249s 100ms/step - loss: 0.6940 - accuracy: 0.5170
Epoch 4/5
2500/2500 [==============================] - 249s 100ms/step - loss: 0.6935 - accuracy: 0.5172
Epoch 5/5
2500/2500 [==============================] - 249s 99ms/step - loss: 0.6935 - accuracy: 0.5178


We can see that we have really bad accuracy lets try something else

In [78]:
inputs = Input(name='inputs',shape=[max])
layer = Embedding(2000,50,input_length=max)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FC1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='out_layer')(layer)
layer = Activation('sigmoid')(layer)
model2 = Model(inputs=inputs,outputs=layer)

In [79]:
model2.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [81]:
history = model2.fit(Xtrain, ytrain, batch_size=80, epochs=6, validation_split=0.1)
print("End")

Epoch 1/6
900/900 [==============================] - 309s 342ms/step - loss: 0.5730 - accuracy: 0.6908 - val_loss: 0.5534 - val_accuracy: 0.7103
Epoch 2/6
900/900 [==============================] - 307s 341ms/step - loss: 0.4977 - accuracy: 0.7588 - val_loss: 0.5128 - val_accuracy: 0.7470
Epoch 3/6
900/900 [==============================] - 306s 340ms/step - loss: 0.4955 - accuracy: 0.7585 - val_loss: 0.5078 - val_accuracy: 0.7508
Epoch 4/6
900/900 [==============================] - 307s 341ms/step - loss: 0.4757 - accuracy: 0.7716 - val_loss: 0.5139 - val_accuracy: 0.7496
Epoch 5/6
900/900 [==============================] - 306s 340ms/step - loss: 0.4723 - accuracy: 0.7736 - val_loss: 0.5128 - val_accuracy: 0.7531
Epoch 6/6
900/900 [==============================] - 307s 342ms/step - loss: 0.4632 - accuracy: 0.7800 - val_loss: 0.5122 - val_accuracy: 0.7514
End


In [83]:
accuracy = model2.evaluate(Xtest,ytest)
print('Test set\n  Accuracy: {:0.2f}'.format(accuracy[1]))

625/625 [==============================] - 34s 55ms/step - loss: 0.4979 - accuracy: 0.7569
Test set
  Accuracy: 0.76
